## Inference images conditioned with prompts

In [ ]:
from tuneavideo.pipelines.pipeline_tuneavideo import TuneAVideoPipeline
from tuneavideo.models.unet import UNet3DConditionModel
from tuneavideo.util import save_videos_grid
import torch

pretrained_model_path = "./checkpoints/CompVis/stable-diffusion-v1-4"
my_model_path = "outputs/supewoman-talking5"
prompt = "a woman, wearing Batman's mask, is talking"

unet = UNet3DConditionModel.from_pretrained(my_model_path, subfolder='unet', torch_dtype=torch.float16).to('cuda')
pipe = TuneAVideoPipeline.from_pretrained(pretrained_model_path, unet=unet, torch_dtype=torch.float16).to("cuda")
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_vae_slicing()

ddim_inv_latent = torch.load(f"{my_model_path}/inv_latents/ddim_latent-500.pt").to(torch.float16)
video = pipe(prompt, latents=ddim_inv_latent, video_length=24, height=512, width=512, num_inference_steps=100, guidance_scale=12.5).videos

save_videos_grid(video, f"./{prompt}.gif")

# Process the generated images (tif) and make new video

In [1]:
from video_process import VideoProcess


name = 'wonderwoman'
original_vid = './data/videos/demo_input.mp4'
generated_gifs = f'./data/gifs/{name}.gif'
vp = VideoProcess(original_vid, generated_gifs)

# get information (PFS, duration, frames) about the input video 
print(vp)
vp.video(f'{name}.mp4') 

Basic information about the input video ./data/videos/demo_input.mp4:
                  FPS: 25.0	Duration: 5.53	                  Total frames: 138
Moviepy - Building video wonderwoman.mp4.
MoviePy - Writing audio in wonderwomanTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video wonderwoman.mp4



Moviepy - Done !
Moviepy - video ready wonderwoman.mp4


### Convert the orginal video to gif (same dim as the generated)

In [ ]:
vp.mp42gifs(outname='input_demo.gif', resize=True)